# 🔬 ISIC Cilt Lezyonu Analizi

**Mustafa Engin Dalgıç | 254309502 | Üsküdar Üniversitesi**

---

## ▶️ Nasıl Çalıştırılır?

1. **Connect** butonuna tıklayın (sağ üst)
2. **Runtime > Run all**
3. Bekleyin! ✅

---

In [ ]:
print("🚀 Program başlıyor...\n")

In [ ]:
# KÜTÜPHANELER
import os, sys, warnings
warnings.filterwarnings('ignore')

try:
    import cv2
except:
    !pip install opencv-python -q
    import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
import zipfile
import shutil

%matplotlib inline

print(f"✅ OpenCV {cv2.__version__}")
print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}\n")

In [ ]:
# PROJE İNDİRME
print("📥 GitHub'dan proje indiriliyor...\n")

# Temizlik
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS"):
    shutil.rmtree("UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
if os.path.exists("project.zip"):
    os.remove("project.zip")

# ZIP indir
url = "https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS/archive/refs/heads/main.zip"
urllib.request.urlretrieve(url, "project.zip")
print("✅ ZIP indirildi")

# ZIP aç
with zipfile.ZipFile("project.zip", 'r') as z:
    z.extractall(".")
print("✅ ZIP açıldı")

# Klasör adını değiştir
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main"):
    shutil.move("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main", "UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
    print("✅ Klasör adı düzeltildi")

# Klasöre geç
os.chdir("UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
print(f"✅ Şu an: {os.getcwd()}\n")

In [ ]:
# VERİ KONTROLÜ
print("🔍 ISIC klasörü kontrol ediliyor...\n")

DATA_PATH = "ISIC"

if os.path.exists(DATA_PATH):
    items = os.listdir(DATA_PATH)
    print(f"✅ ISIC klasörü bulundu")
    print(f"📁 İçinde {len(items)} öğe var\n")
    
    # İlk 10 dosya
    print("İlk 10 öğe:")
    for i, item in enumerate(items[:10], 1):
        print(f"   {i}. {item}")
    if len(items) > 10:
        print(f"   ... ve {len(items)-10} tane daha\n")
else:
    print("❌ ISIC klasörü bulunamadı!\n")

In [ ]:
# GÖRÜNTÜ TARAMA
print("🔍 Görüntü dosyaları taranıyor...\n")

images = []
for root, dirs, files in os.walk(DATA_PATH):
    for f in files:
        if f.lower().endswith(('.jpg', '.jpeg', '.png')):
            images.append(os.path.join(root, f))

print(f"📊 SONUÇ: {len(images)} görüntü bulundu!")

if len(images) > 0:
    print(f"\nİlk 5 görüntü:")
    for i, img in enumerate(images[:5], 1):
        print(f"   {i}. {img}")
else:
    print("❌ Hiç görüntü bulunamadı!")

In [ ]:
# VERİ YÜKLEME
if len(images) == 0:
    print("❌ Görüntü yok, devam edilemiyor!")
else:
    print(f"\n📊 İlk 20 görüntü yükleniyor...\n")
    
    data = []
    for img_path in images[:20]:
        try:
            img = Image.open(img_path)
            w, h = img.size
            ch = 3 if img.mode == 'RGB' else 1
            sz = os.path.getsize(img_path) / 1024
            
            data.append({
                'filename': os.path.basename(img_path),
                'filepath': img_path,
                'width': w,
                'height': h,
                'channels': ch,
                'size_kb': round(sz, 2)
            })
        except:
            pass
    
    df = pd.DataFrame(data)
    print(f"✅ {len(df)} görüntü yüklendi!\n")
    display(df.head())

In [ ]:
# İSTATİSTİKLER
if len(images) > 0 and len(df) > 0:
    print("\n📊 İstatistikler:\n")
    print(f"Çözünürlük:")
    print(f"  Ortalama: {df['width'].mean():.0f}x{df['height'].mean():.0f}")
    print(f"  Min: {df['width'].min()}x{df['height'].min()}")
    print(f"  Max: {df['width'].max()}x{df['height'].max()}")
    print(f"\nBoyut:")
    print(f"  Ortalama: {df['size_kb'].mean():.2f} KB")

In [ ]:
# GÖRÜNTÜ GÖSTERİMİ
if len(images) > 0 and len(df) > 0:
    print("\n🖼️ İlk 3 görüntü gösteriliyor...\n")
    
    def resize(img, size=400):
        h, w = img.shape[:2]
        if max(h,w) > size:
            s = size/max(h,w)
            return cv2.resize(img, None, fx=s, fy=s)
        return img
    
    n = min(3, len(df))
    fig, axes = plt.subplots(n, 2, figsize=(10, n*2.5))
    if n == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n):
        row = df.iloc[i]
        img = cv2.imread(row['filepath'])
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        rgb = resize(rgb)
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
        
        axes[i,0].imshow(rgb)
        axes[i,0].set_title(f"RGB - {row['filename'][:20]}")
        axes[i,0].axis('off')
        
        axes[i,1].imshow(gray, cmap='gray')
        axes[i,1].set_title(f"Grayscale")
        axes[i,1].axis('off')
    
    plt.tight_layout()
    plt.show()
    print("✅ Görüntüler gösterildi!")

In [ ]:
# HİSTOGRAM
if len(images) > 0 and len(df) > 0:
    print("\n📊 Histogram...\n")
    
    row = df.iloc[0]
    img = cv2.imread(row['filepath'])
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb = resize(rgb, 400)
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
    hr = cv2.calcHist([rgb],[0],None,[256],[0,256])
    hg = cv2.calcHist([rgb],[1],None,[256],[0,256])
    hb = cv2.calcHist([rgb],[2],None,[256],[0,256])
    hgray = cv2.calcHist([gray],[0],None,[256],[0,256])
    
    fig, ax = plt.subplots(1,2,figsize=(14,4))
    
    ax[0].plot(hr,'r',alpha=0.7)
    ax[0].plot(hg,'g',alpha=0.7)
    ax[0].plot(hb,'b',alpha=0.7)
    ax[0].set_title('RGB Histogram')
    
    ax[1].plot(hgray,'k')
    ax[1].set_title('Grayscale')
    
    plt.tight_layout()
    plt.show()
    print("✅ Histogram tamamlandı!")

In [ ]:
# CLAHE
if len(images) > 0 and len(df) > 0:
    print("\n✨ CLAHE...\n")
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    fig, ax = plt.subplots(1,2,figsize=(12,5))
    
    ax[0].imshow(gray, cmap='gray')
    ax[0].set_title('Orijinal')
    ax[0].axis('off')
    
    ax[1].imshow(enhanced, cmap='gray')
    ax[1].set_title('CLAHE')
    ax[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    print("✅ CLAHE tamamlandı!")

---

## 🎉 TEST BAŞARILI!

### ✅ Tamamlanan:
- Veri yükleme
- Görüntü gösterimi  
- Histogram
- CLAHE

### 🚀 Tam Program:

In [ ]:
if len(images) > 0:
    print("\n🚀 TAM PROGRAM (10-15 dk)...\n")
    !python isic_project.py

---

**Mustafa Engin Dalgıç** | engindalgic86@gmail.com  
[GitHub](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)